In [1]:
### 한국어 사용하는 LLM 사용해보기 on cpu (gpu memory 6기가밖에 안되가지공,,)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small').to('cpu')

In [2]:
import torch
new_user_input_ids = tokenizer.encode("한국말해봐" + tokenizer.eos_token,return_tensors='pt')
bot_input_ids = new_user_input_ids
result_chat = model.generate(bot_input_ids,max_length = 300, pad_token_id = tokenizer.eos_token_id)

print("DialoGPT: {}".format(tokenizer.decode(result_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure if you're joking or not, but I'm pretty sure it's a joke.


In [3]:
import datasets

dataset = datasets.load_dataset('jiwoochris/easylaw_kr')

In [4]:
print(dataset)
#instruction : 질문
#output : 답변 


DatasetDict({
    train: Dataset({
        features: ['documents'],
        num_rows: 2198
    })
})


In [5]:
from peft import LoraConfig,get_peft_model,get_peft_model_state_dict,set_peft_model_state_dict

lora_r = 8 #rank setting
lora_alpha = 32 #제한
#lora_dropout = 0.05 #dropout설정
lora_target_modules = ['c_attn','c_proj']
print(model.modules)
config = LoraConfig(
    r = lora_r,
    lora_alpha= lora_alpha,
    target_modules= lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model,config)

<bound method Module.modules of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)>


c:\Users\HOME\anaconda3\lib\site-packages\peft\tuners\lora\layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [6]:
train_data = dataset


In [7]:
print(train_data)

DatasetDict({
    train: Dataset({
        features: ['documents'],
        num_rows: 2198
    })
})


In [8]:
num_epochs = 5
batch_size = 2

learning_rate = 1e-05
gradient_accumulation_steps = 2
output_dir = './output'
world_size = 1
ddp = world_size != 1
group_by_length = False
val_set_size = 0
import transformers
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=100,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    bf16=False,
    logging_steps=2,
    evaluation_strategy="steps" if val_set_size > 0 else "no",
    save_strategy="steps",
    eval_steps=200 if val_set_size > 0 else None,
    save_steps=10,
    output_dir=output_dir,
    load_best_model_at_end=True if val_set_size > 0 else False,
    ddp_find_unused_parameters=False if ddp else None,
    report_to="none",
    group_by_length=group_by_length,
)

In [9]:
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer, return_tensors="pt", padding=True)
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return self.tokenizer(
            examples["documents"],
            padding="max_length",
            truncation=True,
        )
special_tokens = tokenizer.special_tokens_map
print(special_tokens)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def add_end_token_to_question(input_dict):
    input_dict['documents'] += special_tokens['bos_token']

    return input_dict

train_data = train_data.map(add_end_token_to_question)
tokenizer_wrapper = TokenizerWrapper(tokenizer)

tokenized_dataset = train_data.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=4, remove_columns=['documents'])
model.resize_token_embeddings(len(tokenizer))


{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}


Embedding(50258, 768)

In [10]:
trainer = transformers.Trainer(
    model = model,
    train_dataset=tokenized_dataset['train'],
    
    args = train_args,
    data_collator=data_collator
)

c:\Users\HOME\anaconda3\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [11]:
trainer.train()

  0%|          | 0/2745 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.